## Run simulation

In [6]:
import itertools as it
import numpy as np
import pandas as pd
import multiprocessing as mp
import time
from tqdm.notebook import tqdm
from game_state import GameState
from blackjack import blackjack_train_agent, blackjack_test_agent, run_blackjack_exp

In [2]:
trials = [10000, 50000, 100000, 250000, 500000, 1000000]
decks = range(1, 7)
etas = np.arange(0.1, 0.9, 0.1)
gammas = np.arange(0.2, 1.0, 0.1)
epss = np.arange(0.1, 0.6, 0.1)

### Serial

In [3]:
param_iter = it.product(decks, trials, epss, etas, gammas)
num_experiments = len(list(param_iter))
# Need to reinitialize
param_iter = it.product(decks, trials, epss, etas, gammas)

exp_data = []
for ndecks, ntrials, eps, eta, gamma in tqdm(param_iter, total=num_experiments):
    #print(ndecks, ntrials, eps, eta, gamma)
    state = GameState(ndecks, 2)
    agent, data = blackjack_train_agent(state, ntrials, eps, eta, gamma)
    exp_data.extend(data)

  0%|          | 0/11520 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Parallel w/progress

In [5]:
param_iter = it.product(decks, trials, epss, etas, gammas)
counter = mp.Value('I', 0)
num_exp = len(list(param_iter))
param_iter = it.product(decks, trials, epss, etas, gammas, [counter], [num_exp])

In [6]:
def run_exp(ndecks, ntrials, eps, eta, gamma, counter, num_exp):
    state = GameState(ndecks, 2)
    _, data = blackjack_train_agent(state, ntrials, eps, eta, gamma)
    
    with counter.get_lock():
        counter.value += 1
        print(f"Progress: {(counter.value + 1) / num_exp * 100:.2f}  %", end='\r')
        
    return data

In [8]:
exp_data = []
with mp.Pool() as pool:
    results = pool.starmap(run_exp, param_iter, chunksize=1000)    

RuntimeError: Synchronized objects should only be shared between processes through inheritance

### Parallel wo/progress

In [9]:
param_iter = it.product(decks, trials, epss, etas, gammas)

In [ ]:
exp_data = []
start = time.time()
with mp.Pool() as pool:
    results = pool.starmap(run_blackjack_exp, param_iter, chunksize=1000)
end = time.time()
print(f"Took {(end - start) / 60} mins")